# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [2]:
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/002_RETAIL'

2.- Nombrar los ficheros de datos.

In [3]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [4]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)

## NUEVAS VARIABLES

En este caso ya nos vienen creadas:

* componentes de la fecha
* variables de calendario

Vamos a crear:

* las que identificamos en EDA
* lags
* ventanas móviles

Para esta fase necesitamos volver a unir en un solo dataframe.

In [5]:
df = pd.concat([cat,num], axis = 1)
df

,weekday,wday,month,year,day_number,item_id,store_id,event_name_1,event_type_1,week_id,ventas,sell_price
date,,,,,,,,,,,,
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_090,CA_3,NewYear,National,11249,0,1.25
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_120,CA_3,NewYear,National,11249,33,1.25
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_202,CA_3,NewYear,National,11249,0,4.98
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_252,CA_3,NewYear,National,11249,0,4.98
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_288,CA_3,NewYear,National,11249,20,4.28
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,Monday,3,11,2015,d_1767,FOODS_3_329,CA_4,Sin_evento,Sin_evento,11544,9,1.68
2015-11-30,Monday,3,11,2015,d_1767,FOODS_3_555,CA_4,Sin_evento,Sin_evento,11544,26,2.48
2015-11-30,Monday,3,11,2015,d_1767,FOODS_3_586,CA_4,Sin_evento,Sin_evento,11544,13,2.48


### Variable demanda intermitente

Esta variable va a identificar cuantos días seguidos han transcurrido con ventas cero.

La definiremos como que si los últimos n días han tenido cero ventas entonces hay rotura de stock.

Podemos crear varias cambiando el n.

Nos servirá para modelizar.

In [6]:
#Función para localizar roturas de stock en los últimos 5 dias. Se puede cambiar n para cambiar la ventana móvil
def rotura_stock(ventas, n = 5):
    cero_ventas = pd.Series(np.where(ventas == 0,1,0))
    num_ceros = cero_ventas.rolling(n).sum()
    rotura_stock = np.where(num_ceros == n,1,0)
    return(rotura_stock)

In [7]:
df = df.sort_values(by = ['store_id','item_id','date'])

In [8]:
df['rotura_stock_3'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x, 3)).values

In [9]:
df['rotura_stock_7'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,7)).values

In [10]:
df['rotura_stock_15'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,15)).values

In [19]:
df.head(10)

,weekday,wday,month,year,day_number,item_id,store_id,event_name_1,event_type_1,week_id,ventas,sell_price,rotura_stock_3,rotura_stock_7,rotura_stock_15
date,,,,,,,,,,,,,,,
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_090,CA_3,NewYear,National,11249,0,1.25,0,0,0
2013-01-15,Tuesday,4,1,2013,d_718,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11251,158,1.25,0,0,0
2013-01-17,Thursday,6,1,2013,d_720,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11251,170,1.25,0,0,0
2013-01-22,Tuesday,4,1,2013,d_725,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,63,1.25,0,0,0
2013-01-23,Wednesday,5,1,2013,d_726,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,0,1.25,0,0,0
2013-01-24,Thursday,6,1,2013,d_727,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,0,1.25,0,0,0
2013-01-25,Friday,7,1,2013,d_728,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,0,1.25,1,0,0
2013-01-28,Monday,3,1,2013,d_731,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11301,127,1.38,0,0,0
2013-01-29,Tuesday,4,1,2013,d_732,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11301,148,1.38,0,0,0


### Variables de lag

Vamos a crear lags sobre las siguientes variables:

* ventas: lags de 15 días
* sell_price: lags de 7 días
* rotura_stock: lag de un día

In [12]:
def crear_lags(df, variable, num_lags = 7):
    
    #Crea el objeto dataframe
    lags = pd.DataFrame()
    
    #Crea todos los lags
    for cada in range(1,num_lags+1):
        lags[variable + '_lag_'+ str(cada)] = df[variable].shift(cada)
    
    #Devuelve el dataframe de lags
    return(lags)

In [13]:
lags_sell_price_df = (df.groupby(['store_id', 'item_id'])
                        .apply(lambda x: crear_lags(df = x, variable = 'sell_price', num_lags= 7), include_groups=False)
                        .reset_index()
                        .set_index('date'))


In [14]:
lags_rotura_stock_3_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_3', num_lags= 1), include_groups=False)
                            .reset_index()
                            .set_index('date'))

In [15]:
lags_rotura_stock_7_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_7', num_lags= 1), include_groups=False)
                            .reset_index()
                            .set_index('date'))

In [16]:
lags_rotura_stock_15_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_15', num_lags= 1), include_groups=False)
                            .reset_index()
                            .set_index('date'))

In [17]:
lags_ventas_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: crear_lags(df = x, variable = 'ventas', num_lags= 15), include_groups=False)
                    .reset_index()
                    .set_index('date'))

In [18]:
df

,weekday,wday,month,year,day_number,item_id,store_id,event_name_1,event_type_1,week_id,ventas,sell_price,rotura_stock_3,rotura_stock_7,rotura_stock_15
date,,,,,,,,,,,,,,,
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_090,CA_3,NewYear,National,11249,0,1.25,0,0,0
2013-01-15,Tuesday,4,1,2013,d_718,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11251,158,1.25,0,0,0
2013-01-17,Thursday,6,1,2013,d_720,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11251,170,1.25,0,0,0
2013-01-22,Tuesday,4,1,2013,d_725,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,63,1.25,0,0,0
2013-01-23,Wednesday,5,1,2013,d_726,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,0,1.25,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-24,Tuesday,4,11,2015,d_1761,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11543,12,1.58,0,0,0
2015-11-27,Friday,7,11,2015,d_1764,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11543,10,1.58,0,0,0
2015-11-28,Saturday,1,11,2015,d_1765,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11544,17,1.58,0,0,0


### Variables de ventanas móviles

Vamos a crear tres tipos de ventanas móviles sobre las ventas:

* mínimo móvil
* media móvil
* máximo móvil

Cada uno de ellos en el rango de 15 días.

In [19]:
def min_movil(df, variable, num_periodos = 7):

    minm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        minm[variable + '_minm_' + str(cada)] = df[variable].shift(1).rolling(cada).min()
    
    #Devuelve el dataframe de lags
    return(minm)

In [20]:
def media_movil(df, variable, num_periodos = 7):

    mm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        mm[variable + '_mm_' + str(cada)] = df[variable].shift(1).rolling(cada).mean()
    
    #Devuelve el dataframe de lags
    return(mm)

In [21]:
def max_movil(df, variable, num_periodos = 7):

    maxm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        maxm[variable + '_maxm_' + str(cada)] = df[variable].shift(1).rolling(cada).max()
    
    #Devuelve el dataframe de lags
    return(maxm)

In [22]:
min_movil_df = (df.groupby(['store_id','item_id'])
                  .apply(lambda x: min_movil(df = x, variable = 'ventas', num_periodos= 15), include_groups=False)
                  .reset_index()
                  .set_index('date'))

In [23]:
media_movil_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: media_movil(df = x, variable = 'ventas', num_periodos= 15), include_groups=False)
                    .reset_index()
                    .set_index('date'))

In [24]:
max_movil_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: max_movil(df = x, variable = 'ventas', num_periodos= 15), include_groups=False)
                    .reset_index()
                    .set_index('date'))

In [25]:
df

,weekday,wday,month,year,day_number,item_id,store_id,event_name_1,event_type_1,week_id,ventas,sell_price,rotura_stock_3,rotura_stock_7,rotura_stock_15
date,,,,,,,,,,,,,,,
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_090,CA_3,NewYear,National,11249,0,1.25,0,0,0
2013-01-15,Tuesday,4,1,2013,d_718,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11251,158,1.25,0,0,0
2013-01-17,Thursday,6,1,2013,d_720,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11251,170,1.25,0,0,0
2013-01-22,Tuesday,4,1,2013,d_725,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,63,1.25,0,0,0
2013-01-23,Wednesday,5,1,2013,d_726,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,0,1.25,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-24,Tuesday,4,11,2015,d_1761,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11543,12,1.58,0,0,0
2015-11-27,Friday,7,11,2015,d_1764,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11543,10,1.58,0,0,0
2015-11-28,Saturday,1,11,2015,d_1765,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11544,17,1.58,0,0,0


## PREPARAR LOS DATASETS

### Unir todos los dataframes generados

In [26]:
df_unido = pd.concat([df,
                      lags_sell_price_df,
                      lags_rotura_stock_3_df,
                      lags_rotura_stock_7_df,
                      lags_rotura_stock_15_df,
                      lags_ventas_df,
                      min_movil_df,
                      media_movil_df,
                      max_movil_df], axis = 1)

# Eliminar columnas duplicadas 
df_unido = df_unido.loc[:,~df_unido.columns.duplicated()]
df_unido

,weekday,wday,month,year,day_number,item_id,store_id,event_name_1,event_type_1,week_id,...,ventas_maxm_6,ventas_maxm_7,ventas_maxm_8,ventas_maxm_9,ventas_maxm_10,ventas_maxm_11,ventas_maxm_12,ventas_maxm_13,ventas_maxm_14,ventas_maxm_15
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,Tuesday,4,1,2013,d_704,FOODS_3_090,CA_3,NewYear,National,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-15,Tuesday,4,1,2013,d_718,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-17,Thursday,6,1,2013,d_720,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-22,Tuesday,4,1,2013,d_725,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-23,Wednesday,5,1,2013,d_726,FOODS_3_090,CA_3,Sin_evento,Sin_evento,11252,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-24,Tuesday,4,11,2015,d_1761,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11543,...,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0
2015-11-27,Friday,7,11,2015,d_1764,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11543,...,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0,17.0
2015-11-28,Saturday,1,11,2015,d_1765,FOODS_3_714,CA_4,Sin_evento,Sin_evento,11544,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0


### Eliminar los nulos que han generado las nuevas variables

In [27]:
#Eliminar los nulos que se generarán al crear los lags por como se construyen las variables. 
#Ej: lag -1 pretende coger los datos del dia anterior al primer dato, por lo que generará un nulo porque no lo tenemos
df_unido.dropna(inplace=True)

### Eliminar las variables que no vamos a necesitar para modelizar

In [33]:
df_unido.columns

Index(['weekday', 'wday', 'month', 'year', 'day_number', 'item_id', 'store_id',
       'event_name_1', 'event_type_1', 'week_id', 'ventas', 'sell_price',
       'rotura_stock_3', 'rotura_stock_7', 'rotura_stock_15',
       'sell_price_lag_1', 'sell_price_lag_2', 'sell_price_lag_3',
       'sell_price_lag_4', 'sell_price_lag_5', 'sell_price_lag_6',
       'sell_price_lag_7', 'rotura_stock_3_lag_1', 'rotura_stock_7_lag_1',
       'rotura_stock_15_lag_1', 'ventas_lag_1', 'ventas_lag_2', 'ventas_lag_3',
       'ventas_lag_4', 'ventas_lag_5', 'ventas_lag_6', 'ventas_lag_7',
       'ventas_lag_8', 'ventas_lag_9', 'ventas_lag_10', 'ventas_lag_11',
       'ventas_lag_12', 'ventas_lag_13', 'ventas_lag_14', 'ventas_lag_15',
       'ventas_minm_2', 'ventas_minm_3', 'ventas_minm_4', 'ventas_minm_5',
       'ventas_minm_6', 'ventas_minm_7', 'ventas_minm_8', 'ventas_minm_9',
       'ventas_minm_10', 'ventas_minm_11', 'ventas_minm_12', 'ventas_minm_13',
       'ventas_minm_14', 'ventas_minm_15', 'ven

In [34]:
a_eliminar = ['day_number','week_id','sell_price','rotura_stock_3','rotura_stock_7','rotura_stock_15']

In [35]:
df_unido.drop(columns=a_eliminar, inplace=True)

### Identificar la target

In [36]:
target = df_unido.ventas

### Separar num y cat

In [37]:
cat = df_unido.select_dtypes(include='O')

In [38]:
num = df_unido.select_dtypes(exclude='O')

## TRANSFORMACIÓN DE CATEGÓRICAS

No vamos a realizar transformaciones numéricas porque no tiene sentido la discretización y la binarización porque nos interesan lo que los datos de las variables numéricas sean las ventas y la normalización y el reescalado no son factores que le afecten a los algoritmos de árboles (LightGBM o XGBoost), por lo que no vamos a hacer esas transformaciones pero, tendríamos que hacer las transformaciones si fueramos a usar una regresión por ejemplo

### One Hot Encoding

#### Variables a aplicar OHE

In [39]:
#Aplicamos OHE en variables como 'year', 'month' o 'wday' porque nos ayuda a que el modelo capture patrones, pero lo ideal
#es probar haciéndole encoding con OHE y OE y ver como predice mejor el modelo

#En el caso en el que hubieramos visto en el EDA que según avanzan los años, meses o dias la variable aumenta o disminuye, 
#entonces tendría sentido codificarla como un OE porque tendrían un sentido de orden
var_ohe = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instanciar

In [40]:
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [41]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [42]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Target Encoding

#### Variables a aplicar TE

In [49]:
#Aplicamos TE en variables como 'year', 'month' o 'wday' porque, en este tipo de proyectos, estas variables pueden ser muy
#predictivas codificadas como TE. De nuevo lo ideal es probar como predice el modelo con esta codificación y sin ella
var_te = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instanciar

In [50]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)

#### Entrenar y aplicar

In [52]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

cat_te = te.fit_transform(cat[var_te], y = target)

#### Guardar como dataframe

In [53]:
#Añadir sufijos a los nombres
nombres_te = [variable + '_te' for variable in var_te]

#Guardar como dataframe
cat_te = pd.DataFrame(cat_te, columns = nombres_te)

## UNIFICAR DATASETS TRANSFORMADOS

### Meter en una lista todos los dataframes generados

Rescatamos de df_unido las variables de segmentación.

In [54]:
de_df_unido = df_unido[['store_id','item_id']].reset_index()

de_df_unido.head(2)

,date,store_id,item_id
0,2013-02-09,CA_3,FOODS_3_090
1,2013-02-10,CA_3,FOODS_3_090


### Unir todos los dataframes

In [55]:
dataframes = [de_df_unido, cat_ohe,cat_te,num.reset_index(drop=True)]

In [56]:
df_tablon = pd.concat(dataframes, axis = 1)

df_tablon

,date,store_id,item_id,year_2013,year_2014,year_2015,month_1,month_2,month_3,month_4,...,ventas_maxm_6,ventas_maxm_7,ventas_maxm_8,ventas_maxm_9,ventas_maxm_10,ventas_maxm_11,ventas_maxm_12,ventas_maxm_13,ventas_maxm_14,ventas_maxm_15
0,2013-02-09,CA_3,FOODS_3_090,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,148.0,148.0,148.0,148.0,148.0,148.0,148.0,170.0,170.0,170.0
1,2013-02-10,CA_3,FOODS_3_090,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,167.0,167.0,167.0,167.0,167.0,167.0,167.0,167.0,170.0,170.0
2,2013-02-11,CA_3,FOODS_3_090,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,167.0,167.0,167.0,167.0,167.0,167.0,167.0,167.0,167.0,170.0
3,2013-02-12,CA_3,FOODS_3_090,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0
4,2013-02-13,CA_3,FOODS_3_090,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16715,2015-11-24,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0
16716,2015-11-27,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0,17.0
16717,2015-11-28,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
16718,2015-11-29,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0


## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [57]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [58]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)